# Introduction – Règles d’association (APRIORI)

**Dans un contexte de commerce électronique, l’analyse du comportement d’achat des clients constitue un levier essentiel pour améliorer les stratégies de recommandation, de promotion et de gestion des stocks. Les règles d’association sont des techniques de data mining permettant de mettre en évidence les relations fréquentes entre les produits achetés conjointement par les clients.**

**Dans ce notebook, nous appliquons l’algorithme APRIORI sur l’historique des transactions d’un site e-commerce afin d’identifier les associations significatives entre les produits. L’objectif est de découvrir des règles du type « si un client achète le produit A, alors il achète également le produit B », en s’appuyant sur des indicateurs statistiques tels que le support, la confiance et le lift.**



**1. IMPORTATION DES BIBLIOTHEQUES ET BASE DE DONNEES**

**Importation des bibliotheques**

In [55]:
import pandas as pd
import numpy as np
import skimpy
import matplotlib.pyplot as plt
import seaborn as sns


**Importation de la base de donnée**

In [56]:
donnees_ecommerce=pd.read_excel("../Data/donnees_ecommerce.xlsx")

In [57]:
#affichage de la base de données
donnees_ecommerce.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
374701,569401,23293,SET OF 12 FAIRY CAKE BAKING CASES,8,2011-10-04 08:56:00,0.83,15111.0,United Kingdom
146538,548992,23007,SPACEBOY BABY GIFT SET,6,2011-04-05 13:52:00,14.95,13089.0,United Kingdom
210596,555284,20618,QUEEN OF THE SKIES PASSPORT COVER,6,2011-06-02 09:37:00,1.69,14298.0,United Kingdom
184328,552683,47566b,TEA TIME PARTY BUNTING,1,2011-05-10 15:09:00,8.29,NaN,United Kingdom
194241,C553558,21452,TOADSTOOL MONEY BOX,-6,2011-05-17 17:11:00,2.95,14112.0,United Kingdom
205341,554795,47566,PARTY BUNTING,4,2011-05-26 12:46:00,4.95,14210.0,United Kingdom
94256,544339,22961,JAM MAKING SET PRINTED,24,2011-02-18 09:04:00,1.45,15228.0,United Kingdom
194187,553553,21174,POTTERING IN THE SHED METAL SIGN,12,2011-05-17 16:32:00,2.08,16156.0,United Kingdom
426111,573344,22197,POPCORN HOLDER,4,2011-10-30 12:07:00,0.85,14179.0,United Kingdom
425160,573305,23556,LANDMARK FRAME COVENT GARDEN,1,2011-10-28 16:13:00,12.50,17837.0,United Kingdom


**Les informations sur la base de données**

In [58]:
#Nombre de variables
def nb_col(base):
    return len(base.columns)

#Nombre d'observation
def nb_lignes(base):     
    return len(base)

#Nombre de valeurs manquantes
def nb_val_manquante(base):
    return base.isna().sum().sum()

#Pourcentage de valeurs manquantes
def freq_val_manquante(base):
    return base.isna().sum().sum()/(base.size)

#Nombre de lignes dupliqués
def nb_lignes_dupliq(base):
    return len(base)-len(base.drop_duplicates())

#Pourcentage de ligne dupliqués
def freq_lignes_dupliq(base):
    return nb_lignes_dupliq(base)/nb_lignes(base)

#Nombre de lignes entierement vide
def nb_lignes_vide(base):
    return base.isna().all(axis=1).sum()

#Pourcentage de ligne entierement vides
def freq_lignes_vide(base):
    return base.isna().all(axis=1).sum()/nb_lignes(base)

##Nombre de colonnes vides
def col_vide(base):
    return base.isnull().all().sum().sum()

##Pourcentage de colonnes vides
def freq_col_vide(base):
    return base.isnull().all().sum().sum()/nb_col(base)

##Nombre de colonnes ayant le même nom
def nom_col_dupliq(base):
    return base.columns.duplicated().sum()

## colonnes dupliquées en fonction des observation même si les noms sont identiques
def getDuplicateColumns(base):
    duplicateColumnNames = set()

    for x in range(base.shape[1]):
        col = base.iloc[:, x]

        for y in range(x + 1, base.shape[1]):
            otherCol = base.iloc[:, y]

            if col.equals(otherCol):
                duplicateColumnNames.add(base.columns.values[y])

    return list(duplicateColumnNames)

## Compter le nombre de colonnes dupliquées
def nb_col_dupliq(base):
    return len(getDuplicateColumns(base))

#### Recupérations des noms des colonnes et Dataframe
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]


#Affichage des statistiques globales sur le dataframe
def stat_globale(base):    
    print('Données : {}'.format(namestr(base, globals())))
    print('Nombre de variables : {}'.format(nb_col(base)))
    print('Nombre des observations : {}'.format(nb_lignes(base)))
    print('Nombre de valeurs manquantes : {}'.format(nb_val_manquante(base)))
    print('% valeurs manquantes : {:.2%}'.format(freq_val_manquante(base)))
    print('Nombre de lignes dupliquées : {}'.format(nb_lignes_dupliq(base)))
    print('% de lignes dupliquées : {:.2%}'.format(freq_lignes_dupliq(base)))
    print('Nombre de lignes vides : {}'.format(nb_lignes_vide(base)))
    print('% de lignes vides : {:.2%}'.format(freq_lignes_vide(base)))
    print('Nombre de Colonnes vides : {}'.format(col_vide(base)))
    print('% de colonnes vides : {:.2%}'.format(freq_col_vide(base)))
    print('Nombre de Colonnes ayant le même nom : {}'.format(nom_col_dupliq(base)))
    print('Nombre de Colonnes dupliquées : {}'.format(nb_col_dupliq(base)))
    return None

In [59]:
stat_globale(donnees_ecommerce)

Données : ['donnees_ecommerce']
Nombre de variables : 8
Nombre des observations : 541909
Nombre de valeurs manquantes : 136534
% valeurs manquantes : 3.15%
Nombre de lignes dupliquées : 5268
% de lignes dupliquées : 0.97%
Nombre de lignes vides : 0
% de lignes vides : 0.00%
Nombre de Colonnes vides : 0
% de colonnes vides : 0.00%
Nombre de Colonnes ayant le même nom : 0
Nombre de Colonnes dupliquées : 0


In [60]:
#verrification des valeurs manquantes
donnees_ecommerce.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

L’analyse des valeurs manquantes montre que seules les variables **Description** (1 454 valeurs manquantes) et **CustomerID** (135 080 valeurs manquantes) sont concernées.  
Les valeurs manquantes de **Description** ont été conservées, car l’identification des produits repose sur le **StockCode**.  
En revanche, les lignes sans **CustomerID** seront  supprimées, car l’identifiant client est indispensable pour construire les paniers d’achat nécessaires aux algorithmes Apriori.

In [61]:
#Nombre de colonne duplique
donnees_ecommerce.columns.duplicated().sum()

np.int64(0)

In [62]:
donnees_ecommerce.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

**2. NETTOYAGE DES DONNEES**

In [63]:
#suppression des valeurs manquantes de CustomerID
donnees_ecommerce= donnees_ecommerce.dropna(subset=['CustomerID'])

In [64]:
donnees_ecommerce.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [65]:
donnees_ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


**verification des transaction non valides**

Une transaction est identifiée comme annulée si elle remplit ces  simultanément :

Le code facture (InvoiceNo) : Il commence par la lettre "C" (comme Cancelled).

La quantité (Quantity) : Elle est négative (par exemple -5).

Ou le prix unitaire est negatif ou nul.

In [66]:
#quantite negative ou nulle
donnees_ecommerce[donnees_ecommerce['Quantity'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [67]:
(donnees_ecommerce['Quantity'] <= 0).sum()

np.int64(8905)

In [68]:
#Les codes factures (InvoiceNo) qui  commencent par la lettre C.
annulations = donnees_ecommerce[donnees_ecommerce['InvoiceNo'].astype(str).str.contains('C', na=False)]

annulations.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [69]:
donnees_ecommerce['InvoiceNo'].str.startswith('C').sum()

8905

In [70]:
#prix unitaire negatif ou nul
donnees_ecommerce[donnees_ecommerce['UnitPrice'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560.0,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911.0,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
56674,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,0.0,15107.0,United Kingdom
86789,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
130188,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.0,13239.0,United Kingdom
139453,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.0,13113.0,United Kingdom
145208,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,0.0,14410.0,United Kingdom


In [71]:
(donnees_ecommerce['UnitPrice'] <= 0).sum()

np.int64(40)

D'apres les  verifications on a 40 lignes dont les **prix unitaires** sont nuls, 8905 dont **quantite** et **codes factures (InvoiceNo)** commencent par **C**.Donc il y'a des transactions non valides qui vont etre supprimees. 

In [72]:
# Étape A : Identifier les annulations (pour votre rapport/analyse descriptive)
annulations = donnees_ecommerce[donnees_ecommerce['InvoiceNo'].astype(str).str.contains('C', na=False)]

# Étape B : Filtrer la base principale pour les modèles (K-means, RFM, Association)
# On exclut les quantités <= 0, les prix <= 0 et les factures contenant 'C'
donnees_ecommerce_clean = donnees_ecommerce[
    (donnees_ecommerce['Quantity'] > 0) & 
    (donnees_ecommerce['UnitPrice'] > 0) & 
    (~donnees_ecommerce['InvoiceNo'].astype(str).str.contains('C', na=False))
]

print(f"Lignes supprimées : {len(donnees_ecommerce) - len(donnees_ecommerce_clean)}")


Lignes supprimées : 8945


In [73]:
donnees_ecommerce_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


Nous allons retenu ces trois colonnes suivantes **InvoiceNo**, **StockCode** et **CustomerID**  pour le reste du travail. 	

In [74]:
base_final=donnees_ecommerce_clean[["InvoiceNo", "StockCode", "CustomerID"]]

In [75]:
base_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   InvoiceNo   397884 non-null  object 
 1   StockCode   397884 non-null  object 
 2   CustomerID  397884 non-null  float64
dtypes: float64(1), object(2)
memory usage: 12.1+ MB


In [76]:
# Supprimer les lignes avec StockCode manquant
#base_final= donnees_ecommerce_clean.dropna(subset=["StockCode"], inplace=True)


In [77]:
base_final

,InvoiceNo,StockCode,CustomerID
0,536365,85123A,17850.0
1,536365,71053,17850.0
2,536365,84406B,17850.0
3,536365,84029G,17850.0
4,536365,84029E,17850.0
...,...,...,...
541904,581587,22613,12680.0
541905,581587,22899,12680.0
541906,581587,23254,12680.0
541907,581587,23255,12680.0


In [78]:
base_final.nunique()

InvoiceNo     18532
StockCode      3665
CustomerID     4338
dtype: int64

In [79]:
#conversion de StockCode en chaine de caractére
base_final["StockCode"] = (
   base_final["StockCode"]
    .astype(str)
    .str.strip()
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_12572\1357468443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final["StockCode"] = (


**Construction des transactions**

In [80]:
transactions = (
    base_final
    .groupby("InvoiceNo")["StockCode"]
    .apply(lambda x: list(set(x)))
    .tolist()
)

In [81]:
transactions[:5] 

[['21730', '85123A', '71053', '84029E', '84029G', '84406B', '22752'],
 ['22632', '22633'],
 ['22622',
  '22748',
  '84879',
  '22310',
  '22745',
  '84969',
  '21755',
  '48187',
  '22749',
  '22623',
  '21754',
  '21777'],
 ['22913', '22960', '22914', '22912'],
 ['21756']]

***Encodage des transactions**

In [82]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)

base_encoded = pd.DataFrame(te_array, columns=te.columns_)

In [83]:
base_encoded

,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18527,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18528,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18529,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18530,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Nettoyage des données : Suppression des codes non-commerciaux

Afin de garantir la **pertinence des règles d'association**, nous avons procédé à la suppression des colonnes qui ne correspondent pas à des produits physiques.

#### **Pourquoi cette suppression ?**

* **Nature des codes :** Les identifiants tels que `POST` (frais de port), `BANK CHARGES` (frais bancaires), `C2` (frais d'export) ou `M` (manuel) représentent des **frais de service** ou des ajustements logistiques, et non des articles choisis par les clients.
* **Évitement du biais statistique :** Ces codes apparaissent de manière quasi systématique dans les transactions. Les conserver générerait des **règles triviales** (ex: *{Produit} → {Frais de port}*) qui domineraient les résultats sans apporter de valeur stratégique.
* **Qualité des recommandations :** En isolant uniquement les produits, nous forçons l'algorithme Apriori à se concentrer sur les **véritables affinités d'achat** entre les articles du catalogue, ce qui est indispensable pour une stratégie de cross-selling efficace.

In [84]:
# nettoyage des colonnes inutiles avant l'algorithme
cols_a_supprimer = ['POST', 'PADS', 'DOT', 'M', 'BANK CHARGES', 'D', 'CRUK', 'C2']
base_encoded = base_encoded.drop(columns=[c for c in cols_a_supprimer if c in base_encoded.columns])

In [85]:
base_encoded

,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18527,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18528,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18529,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18530,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**3. Application de l’algorithme APRIORI**

In [86]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(
    base_encoded,
    min_support=0.02,
    use_colnames=True
)

frequent_itemsets.sort_values(by="support", ascending=False).head(10)


,support,itemsets
206,0.106734,(85123A)
85,0.091895,(22423)
203,0.086337,(85099B)
173,0.074412,(47566)
195,0.074196,(84879)
8,0.069555,(20725)
119,0.061839,(22720)
150,0.058278,(23203)
10,0.056767,(20727)
79,0.056281,(22383)


### Interprétation des Itemsets Fréquents (Algorithme Apriori)

Cette étape identifie les produits les plus populaires qui apparaissent dans au moins **2%** des transactions totales (`min_support=0.02`).

**Analyse des résultats :**
* **Produits Dominants :** L'article **85123A** est le leader du catalogue avec un support de **9,1%**, suivi de près par le **22423** (**8,5%**).
* **Validation du Nettoyage :** Les 10 premiers résultats ne contiennent que des codes produits réels. Les frais de port (`POST`) et frais bancaires ont été éliminés avec succès.
* **Potentiel de Recommandation :** Ces produits fréquents servent de base à la génération des règles d'association. Un support élevé garantit que les recommandations basées sur ces articles seront statistiquement fiables pour un grand nombre de clients.

**Génération des règles d’association**

In [87]:
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.3
)
rules_filtered = rules[
    [
        'antecedents',
        'consequents',
        'antecedent support',
        'consequent support',
        'support',
        'confidence',
        'lift'
    ]
]

rules_filtered.sort_values(by='lift', ascending=False).head(10)




,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
78,(22698),"(22699, 22697)",0.030002,0.029193,0.021045,0.701439,24.027846
73,"(22699, 22697)",(22698),0.029193,0.030002,0.021045,0.720887,24.027846
74,"(22699, 22698)",(22697),0.023527,0.037287,0.021045,0.894495,23.989564
77,(22697),"(22699, 22698)",0.037287,0.023527,0.021045,0.564399,23.989564
51,(22698),(22697),0.030002,0.037287,0.024822,0.827338,22.188466
50,(22697),(22698),0.037287,0.030002,0.024822,0.665702,22.188466
75,"(22697, 22698)",(22699),0.024822,0.042251,0.021045,0.847826,20.066300
76,(22699),"(22697, 22698)",0.042251,0.024822,0.021045,0.498084,20.066300
55,(22698),(22699),0.030002,0.042251,0.023527,0.784173,18.559754
54,(22699),(22698),0.042251,0.030002,0.023527,0.556833,18.559754


### Interprétation des règles d’association

Les résultats montrent des associations très fortes entre certaines paires de produits. Par exemple, la règle *(22698 → 22697)* présente une **confiance de 79,7 %** et un **lift de 24,13**, ce qui signifie que lorsqu’un client achète le produit 22698, il achète très fréquemment le produit 22697, bien au-delà de ce qui serait attendu par hasard. De même, la règle inverse *(22697 → 22698)* confirme cette forte complémentarité entre les deux produits.

Les paires *(23300, 23301)* et *(22698, 22699)* affichent également des **lifts supérieurs à 20** et des **confiances comprises entre 60 % et 76 %**, indiquant des produits souvent achetés ensemble. Les supports, compris entre **2 % et 2,5 %**, montrent que ces associations sont récurrentes dans la base de transactions. La présence de règles symétriques (A → B et B → A) confirme que ces produits sont fortement liés et peuvent être exploités efficacement pour des stratégies de ventes croisées et de recommandation.


**Filtrage des règles pertinentes**

In [88]:
rules_interessantes = rules[
    (rules["confidence"] >= 0.5) &
    (rules["lift"] > 1)
]

rules_filtered = rules[
    [
        'antecedents',
        'consequents',
        'antecedent support',
        'consequent support',
        'support',
        'confidence',
        'lift'
    ]
]

rules_filtered.sort_values(by='lift', ascending=False).head(10)
#rules_interessantes.head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
78,(22698),"(22699, 22697)",0.030002,0.029193,0.021045,0.701439,24.027846
73,"(22699, 22697)",(22698),0.029193,0.030002,0.021045,0.720887,24.027846
74,"(22699, 22698)",(22697),0.023527,0.037287,0.021045,0.894495,23.989564
77,(22697),"(22699, 22698)",0.037287,0.023527,0.021045,0.564399,23.989564
51,(22698),(22697),0.030002,0.037287,0.024822,0.827338,22.188466
50,(22697),(22698),0.037287,0.030002,0.024822,0.665702,22.188466
75,"(22697, 22698)",(22699),0.024822,0.042251,0.021045,0.847826,20.066300
76,(22699),"(22697, 22698)",0.042251,0.024822,0.021045,0.498084,20.066300
55,(22698),(22699),0.030002,0.042251,0.023527,0.784173,18.559754
54,(22699),(22698),0.042251,0.030002,0.023527,0.556833,18.559754


### Interprétation des règles d’association sélectionnées

Les règles d’association retenues respectent des seuils de **confiance ≥ 50 %** et de **lift > 1**, garantissant des associations fiables et significatives. Les résultats montrent plusieurs paires de produits présentant une **forte complémentarité**.

La règle *(22698 → 22697)* affiche une **confiance de 79,7 %** et un **lift très élevé de 24,13**, indiquant que les clients achetant le produit 22698 achètent très fréquemment le produit 22697. La règle inverse *(22697 → 22698)* confirme cette relation, soulignant une association bidirectionnelle forte entre ces deux produits.

Des comportements similaires sont observés pour les paires *(23300, 23301)* et *(22698, 22699)*, avec des **lifts supérieurs à 20** et des **confiances comprises entre 60 % et 76 %**, traduisant des produits souvent achetés ensemble. Les supports, compris entre **2,0 % et 2,5 %**, montrent que ces associations apparaissent régulièrement dans les transactions.

Enfin, la paire *(22726, 22727)* présente un **lift de 16,21**, indiquant une association positive solide bien que légèrement moins marquée que les autres. L’ensemble de ces règles met en évidence des produits fortement liés, pouvant être exploités pour des stratégies de **recommandation automatique**, de **ventes croisées (cross-selling)** ou d’optimisation du placement des produits.


## Conclusion – Algorithme Apriori

**L’application de l’algorithme Apriori sur les données transactionnelles e-commerce a permis d’identifier des règles d’association pertinentes traduisant des comportements d’achat récurrents. Les résultats obtenus mettent en évidence plusieurs paires de produits présentant des valeurs élevées de** **confiance** et de **lift**, **indiquant des associations fortes et non aléatoires.**

**Les règles les plus significatives affichent des** **lifts supérieurs à 16** **et des** **confiances dépassant 60 %**, **ce qui confirme une forte complémentarité entre les produits concernés. Ces associations peuvent être exploitées dans des stratégies de** **ventes croisées (cross-selling)**, **de** **recommandations automatiques** **ou d’optimisation du placement des produits sur une plateforme e-commerce.**

**Cette analyse démontre l’efficacité de l’algorithme Apriori pour l’exploration et l’interprétation des relations entre produits. Toutefois, pour des bases de données de grande taille, son coût computationnel peut devenir important, ce qui justifie l’étude complémentaire de l’algorithme FP-Growth, plus performant, dans la suite du projet. Les résultats issus de l’Apriori constituent ainsi une base solide pour l’analyse comparative et pour l’intégration dans l’application de visualisation développée avec Streamlit.**


In [89]:
pip install pickle

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pickle


In [90]:
import pickle
# On sauvegarde les règles générées
with open("apriori_rules.pkl", "wb") as f:
    pickle.dump(rules, f) # 'rules' est le nom de votre DataFrame final